### Define helper functions

In [51]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
%matplotlib inline

In [52]:
pd.options.display.width = 250
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 250


In [53]:
# some helper functions to obtain the right output from API
def remove_spaces(input):
    return input.replace(' ', '%20')

def pdf_to_text(pdf_path):
    #sys.setdefaultencoding("utf-8")
    os_encoding = locale.getpreferredencoding()
    args = ["pdftotext.exe",pdf_path, "-"]
    res = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = (res.stdout).decode(os_encoding,'ignore')
    return(output)


# this is the preferred method, pdfplumber works better then pdftotext
def pdf_to_text2(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        return first_page.extract_text(x_tolerance=2, y_tolerance=3)


def retrieve_and_save_pdf(id, overwrite=False):
    if not os.path.exists(path + id + '.pdf') or overwrite:
        base = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
        url = 'Document('+id+')/Resource'
        end = ''
        response = urllib.request.urlopen(remove_spaces(base+url))
        data = response.read()      # a `bytes` object
        f = open(path+id+'.pdf', 'w+b')
        f.write(data)
        f.close()
    else:
        pass

def query_API(skip):
    base = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
    url = "/Zaak?$filter= Soort eq 'Motie'&$expand=Besluit($expand=Stemming),Document,ZaakActor,Agendapunt&$count=true&$skip="+skip
    end = '&$format=application/json;odata.metadata=full'
    response = urllib.request.urlopen(remove_spaces(base+url))
    data = response.read()      # a `bytes` object
    return(json.loads(data))
#retrieve_and_save_pdf('1b7f8f8c-4579-4103-9f51-9280358f7b8a')
#a = pdf_to_text(path+'3c2dbc59-beb1-42d8-9be4-e09238f20a97.pdf')
#print(a)

### Process API response to dict

In [4]:
def add_API_to_dict(data):
    try:
        for z in data['value']:
            #print('\n\n','nummer',z['Nummer'])
            info[z['Nummer']]={'Titel':z['Titel'],
                              #'Status':z['Status'],
                              'Onderwerp':z['Onderwerp'],
                              'Vergaderjaar':z['Vergaderjaar'],
                              'GestartOp':z['GestartOp']}

            for b in z['Besluit']:
                info[z['Nummer']]['BesluitTekst']=b['BesluitTekst']
                info[z['Nummer']]['StemmingsSoort']=b['StemmingsSoort']
                info[z['Nummer']]['BesluitSoort']=b['BesluitSoort']

                #print('\n','besluit',b['BesluitTekst'])
                if b['BesluitSoort'] and b['BesluitSoort'] in ['Stemmen - aangenomen','Stemmen - verworpen', 'Stemmen - niet aangenomen']:        
                    for s in b['Stemming']:
                        #print(s)
                        if s['Soort'] in ['Voor', 'Tegen']:
                            info[z['Nummer']]['Stem_'+s['ActorFractie']]=(s['Soort'])
                            info[z['Nummer']]['Aantal_stemmen_'+str(s['ActorFractie'])]=(s['FractieGrootte'])
                        #info[z['Nummer']]['Vergissing_'+str(s['ActorFractie'])]=(s['Vergissing']) niet nodig, want Stem is altijd de gecorrigeerde stem (dus niet de vergissing)
                    #print('stemmingenverwerkt')
                    break
            for a in z['Agendapunt']:
                info[z['Nummer']]['AgendapuntOnderwerp']=a['Onderwerp']

            for d in z['Document']:
                if d['Soort'][:5]=='Motie':
                    #print('\n','doc',d)
                    info[z['Nummer']]['doc_Id']=d['Id']
                    info[z['Nummer']]['Volgnummer']=d['Volgnummer']
                    info[z['Nummer']]['Datum']=d['Datum']
                    if INCLUDE_PDF:
                        #print('doing pdf work')
                        retrieve_and_save_pdf(d['Id'])
                        info[z['Nummer']]['Text']=pdf_to_text(path+d['Id']+'.pdf')
            
            info[z['Nummer']]['Indiener_persoon'] = []
            info[z['Nummer']]['Medeindiener_persoon'] = []
            for za in z['ZaakActor']:
                #print('\n','za',za)
                if za['Relatie'] == 'Indiener' and za['ActorFractie']:
                    info[z['Nummer']]['Indiener_persoon'].append(za['ActorNaam'])
                    info[z['Nummer']]['Indiener_'+za['ActorFractie']]=1
                
                if za['Relatie'] == 'Medeindiener' and za['ActorFractie']:
                    info[z['Nummer']]['Medeindiener_persoon'].append(za['ActorNaam'])
                    info[z['Nummer']]['Medeindiener_'+za['ActorFractie']]=1
    except Exception as e: 
        print(e)
        pprint.pprint(z) 
        traceback.print_exc() 
        print(s)
        sys.exit()


### Get all moties

In [5]:
# main routine to query api
FIRST_ONLY = False
INCLUDE_PDF = False
START = 0
MAXIMUM = 100000
path = 'pdf/' # where to store pdfs of motions
skip = 0
info = {}
print ('started')
data = query_API(str(START))
count = data['@odata.count']
# add_API_to_dict(data)
skip = START +  250

started


In [6]:
if not FIRST_ONLY:
    while skip < count + 250 and skip < MAXIMUM:
        print('query for ',skip)
        data = query_API(str(skip))
        add_API_to_dict(data)
        skip +=250
print('finished')

query for  250
query for  500
query for  750
query for  1000
query for  1250
query for  1500
query for  1750
query for  2000
query for  2250
query for  2500
query for  2750
query for  3000
query for  3250
query for  3500
query for  3750
query for  4000
query for  4250
query for  4500
query for  4750
query for  5000
query for  5250
query for  5500
query for  5750
query for  6000
query for  6250
query for  6500
query for  6750
query for  7000
query for  7250
query for  7500
query for  7750
query for  8000
query for  8250
query for  8500
query for  8750
query for  9000
query for  9250
query for  9500
query for  9750
query for  10000
query for  10250
query for  10500
query for  10750
query for  11000
query for  11250
query for  11500
query for  11750
query for  12000
query for  12250
query for  12500
query for  12750
query for  13000
query for  13250
query for  13500
query for  13750
query for  14000
query for  14250
query for  14500
query for  14750
query for  15000
query for  15250
query

In [7]:
import pickle
with open('moties_unprocessed.pickle', 'wb') as handle:
    pickle.dump(info, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Download PDFs

In [8]:
import pickle
file = open("moties_unprocessed.pickle","rb")
info = pickle.load(file)

download_errors = set()
no_doc_id = set()

# takes a long time
for i, (k,v) in enumerate(info.items()):
    if i % 100 == 0: print(i)
    doc_id = v.get('doc_Id', None)
    if doc_id:
        try:
            retrieve_and_save_pdf(doc_id)
            info[k]['Text'] = pdf_to_text2(path + doc_id +'.pdf')
        except Exception as e:
            print('download or pdf_to_text error')
            traceback.print_exc() 
            download_errors.add((k,doc_id))
    else:
        no_doc_id.add(k)
df_text = pd.DataFrame(info).T

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

Traceback (most recent call last):
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\1447062882.py", line 15, in <module>
    info[k]['Text'] = pdf_to_text2(path + doc_id +'.pdf')
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\3182530635.py", line 16, in pdf_to_text2
    with pdfplumber.open(pdf_path) as pdf:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 78, in open
    return cls(
           ^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 40, in __init__
    self.doc = PDFDocument(PDFParser(stream), password=password)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Root object! - Is this really a PDF?")
pdfminer.p

24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
download or pdf_to_text error


Traceback (most recent call last):
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\1447062882.py", line 15, in <module>
    info[k]['Text'] = pdf_to_text2(path + doc_id +'.pdf')
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\3182530635.py", line 16, in pdf_to_text2
    with pdfplumber.open(pdf_path) as pdf:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 78, in open
    return cls(
           ^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 40, in __init__
    self.doc = PDFDocument(PDFParser(stream), password=password)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Root object! - Is this really a PDF?")
pdfminer.p

34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
download or pdf_to_text error


Traceback (most recent call last):
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\1447062882.py", line 15, in <module>
    info[k]['Text'] = pdf_to_text2(path + doc_id +'.pdf')
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\3182530635.py", line 16, in pdf_to_text2
    with pdfplumber.open(pdf_path) as pdf:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 78, in open
    return cls(
           ^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 40, in __init__
    self.doc = PDFDocument(PDFParser(stream), password=password)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Root object! - Is this really a PDF?")
pdfminer.p

39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100
43200
43300
43400
43500
43600
43700
43800
43900
44000
44100
44200
44300
44400
44500
44600
44700
44800
44900
45000
45100
45200
45300
45400
45500
45600
45700
45800
45900
46000
46100
46200
46300
46400
46500
46600
46700
46800
46900
47000
47100
47200
47300
47400
47500
47600
47700
47800
47900
48000
48100
48200
48300
48400
48500
48600
48700
48800
48900
49000
download or pdf_to_text error


Traceback (most recent call last):
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\1447062882.py", line 15, in <module>
    info[k]['Text'] = pdf_to_text2(path + doc_id +'.pdf')
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_9276\3182530635.py", line 16, in pdf_to_text2
    with pdfplumber.open(pdf_path) as pdf:
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 78, in open
    return cls(
           ^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfplumber\pdf.py", line 40, in __init__
    self.doc = PDFDocument(PDFParser(stream), password=password)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Root object! - Is this really a PDF?")
pdfminer.p

49100
49200
49300
49400
49500
49600
49700
49800
49900
50000
50100
50200
50300
50400
50500
50600
50700
50800
50900
51000
51100
51200
51300
51400
51500
51600


In [11]:
# Some pdf download may have been corrupted. By running this function, we try to redownload the pdf and update the text. Unfortunatly it seems the pdfs are simply corrupted
no_text_idx = df_text[(df_text['Text']=='') | (df_text['Text'].isna())].index
print(len(no_text_idx), 'no texts found')
texts = []
for index in no_text_idx:
    doc_Id = df_text.loc[index,'doc_Id']
    print(doc_Id, type(doc_Id))
    if not pd.isna(doc_Id):
        retrieve_and_save_pdf(doc_Id, overwrite=True)
        t = pdf_to_text(path + doc_Id + '.pdf')
        if t != '':
            df_text.loc[index,'Text'] = t
            texts.append(t)
print(len(texts), 'new pdfs found')

1 no texts found
aed6d3ce-2ebb-4438-a6e8-9d0b344672f4 <class 'str'>
0 new pdfs found


In [12]:
import pickle
with open('df_text_before_preprocessing', 'wb') as handle:
    pickle.dump(df_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Make dataframe

In [54]:
import pickle
file = open("df_text_before_preprocessing","rb")
df = pickle.load(file)
print(len(df))
column_list = df.columns.values # current columns

51663


In [55]:
df.iloc[0]['Text']

'2\nTweede Kamer der Staten-Generaal\nVergaderjaar 2014–2015\n34 000 XIII Vaststelling van de begrotingsstaten van het\nMinisterie van Economische Zaken (XIII) en het\nDiergezondheidsfonds (F) voor het jaar 2015\nNr. 103 MOTIE VAN HET LID AGNES MULDER\nVoorgesteld tijdens het Wetgevingsoverleg van 17 november 2014\nDe Kamer,\ngehoord de beraadslaging,\noverwegende dat er vraag is naar de techniek van monomestvergisting bij\nagrarische\nondernemers en dat deze techniek veel potentieel heeft voor emissiebe-\nsparing en duurzame-energie-opwek;\nverzoekt de regering om te bekijken hoe eenmalig investeringen in\nmonomestvergisting extra ondersteund kunnen worden om een grotere\ntoepassing en kostprijsverlaging van deze installaties te bereiken,\nen gaat over tot de orde van de dag.\nAgnes Mulder\nkst-34000-XIII-103ISSN\n0921 - 7371\n’s-Gravenhage 2014 Tweede Kamer, vergaderjaar 2014–2015, 34 000 XIII, nr. 103'

### Preprocessing

In [56]:
def preprocessing_motie(text,doc_id,i): # remove start and end of document to only include relevant text
    try:
        if text and doc_id and text != 'nan':
            try:
                regex = re.findall(r"\d{4}(.*?)Nr.*gehoord de beraadslaging(.*)",text,re.DOTALL)
                regex = (' '.join(regex[0]))
            except Exception:
                regex = re.findall(r"\d{4}(.*?)",text,re.DOTALL)
                regex = (' '.join(regex[0]))
            return regex
        else:
            if not doc_id:
                doc_id.append(i)
            elif not text:
                no_text.append(i)
            return ""
    except Exception:
        print(traceback.print_exc())
        error_processing.append(i)
        return ""


In [57]:
# df['Text'] = df['Text'].astype(str)
# df['Text'] = df['Text'].str.replace('\xad', '')
# df['Text'] = df['Text'].str.replace('\n', '')
# df['Text'] = df['Text'].str.replace('\r', '')
# df['Text'] = df['Text'].str.replace('-', '')

In [58]:
# forget about text preprocessing, do that in later notebook with LDA
import numpy as np
no_text = []
error_processing = []
doc_id = []
# df['Text_processed'] = np.vectorize(preprocessing_motie)(df['Text'],df['doc_Id'],df.index)
len(no_text),len(error_processing),len(doc_id)

(0, 0, 0)

In [59]:
#verwijder hoofdelijke stemmingen
print (len(df), 'removing hoofdelijke stemmingen')
df.drop(df[df['StemmingsSoort'] == 'Hoofdelijk'].index, inplace=True)
print (len(df), '\n')

#verwijder moties zonder stemming
print (len(df), 'removing moties without vote')
print(df['BesluitSoort'].value_counts())
df = df[((df['BesluitSoort'] == 'Stemmen - aangenomen') | (df['BesluitSoort'] == 'Stemmen - verworpen'))]
print (len(df), '\n')

#recode besluitsoort naar -1 en 1
print(df['BesluitSoort'].value_counts())
df['BesluitSoort']=df['BesluitSoort'].replace({'Stemmen - verworpen':'0','Stemmen - aangenomen':'1'})
df['BesluitSoort']=pd.to_numeric(df['BesluitSoort'])
print(df['BesluitSoort'].value_counts(), '\n')

#recode voor en tegen naar 1 en -1
stem_column = [c for c in column_list if 'Stem_' in c]
print(df['Stem_50PLUS'].value_counts())
df[stem_column]=df[stem_column].replace({'Tegen':'-1','Voor':'1','Niet deelgenomen':np.nan})
print(df['Stem_50PLUS'].value_counts(), '\n')

#cast to datetime and sort old to new
df['Datum'] = pd.to_datetime(df['Datum'].str[:10]) # cut off the time & timezone info, we only need date
# df['Datum'] = pd.to_datetime(df['Datum'], utc=True)
df['GestartOp'] = pd.to_datetime(df['GestartOp'].str[:10])
# df['GestartOp'] = pd.to_datetime(df['GestartOp'], utc=True)
# df['Datum'] = df['Datum'].dt.tz_convert(None)
# df['GestartOp'] = df['GestartOp'].dt.tz_convert(None)
df.sort_values('Datum',inplace=True)

51663 removing hoofdelijke stemmingen
51363 

51363 removing moties without vote
BesluitSoort
Stemmen - verworpen                                       21117
Stemmen - aangenomen                                      19037
Ingediend                                                  4965
Stemmen - aangehouden                                      2533
Termijn - vervallen in verband met verstrijken termijn     1363
Stemmen - aangehouden (al tijdens debat)                    729
Stemmen - ingetrokken                                       563
Overgenomen                                                 388
Stemmen - ingetrokken (al tijdens debat)                    232
Stemmen - uitstellen                                        126
Agenderen - stemmingen                                       10
[Vrij tekstveld / geen Parlisproces]                          4
Ter informatie                                                4
Behandeling wordt voortgezet                                  2
Stemmen   

In [60]:
column_list = df.columns.values
indiener_column = [c for c in column_list if ('Indiener') in c and c != 'Indiener_persoon']
# make a column with the indienende partij
ind = pd.melt(df, value_vars=indiener_column, ignore_index=False)
ind = ind[ind['value'] == 1]
ind = ind.rename(columns={'variable':'Indienende_partij'})
ind['Indienende_partij'] = ind['Indienende_partij'].str[9:]
df['Indiener_persoon'] = df['Indiener_persoon'].map(lambda x: x[0] if x else '')
df = df.join(ind['Indienende_partij'], how='left')
df['Indienende_persoon_partij'] = df['Indiener_persoon'] + ', ' + df['Indienende_partij']

In [61]:
#bereken voor en tegenstemmen
aantal_stemmen_column = [c for c in column_list if 'Aantal' in c]

for i in range(len(stem_column)):
    df[stem_column[i]]=pd.to_numeric(df[stem_column[i]])
    df[aantal_stemmen_column[i]]=pd.to_numeric(df[aantal_stemmen_column[i]])

res = pd.DataFrame(np.multiply(df[stem_column].fillna(0).values, df[aantal_stemmen_column].fillna(0).values))
voor = res[res > 0].sum(axis=1)
tegen = abs(res[res < 0].sum(axis=1))
df['Voor'], df['Tegen'] = voor.values,tegen.values
df['Delta'] = abs(df['Voor']- df['Tegen'])
df['Sum'] = (df['Voor']+ df['Tegen'])


In [62]:
df = df.replace(-1,0)

In [63]:
df['Jaar'] = df['Datum'].dt.year

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2788\483433239.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Jaar'] = df['Datum'].dt.year


In [64]:
# only 26 moties for first year, so let's chop
df = df[(df['Jaar'] > 2008) & (df['Jaar'] < 2025)]

In [65]:
from collections import namedtuple
import datetime
Cabinet = namedtuple('Cabinet', ['name','start', 'end', 'demissionair','coalition'])
Chamber = namedtuple('Chamber', ['name','start', 'end', 'numyears'])

chamber_input = [
    "Balkenende IV,01 01 2009,16 06 2010",# mark begin 2008 is not correct (should be 30 11 2006) but for dataset purpose it it
    "Rutte I,17 06 2010,19 09 2012",
    "Rutte II,20 09 2012,22 03 2017",
    "Rutte III,23 03 2017,30 03 2021", # mark end 2020 is not correct but for dataset purpose it it
    "Rutte IV,31 03 2021,31 12 2024"] 

chambers = {}
for c in chamber_input:
    c = c.split(',')
    name = c[0]
    start = list(map(int, c[1].split()))
    startdate = datetime.datetime(start[2], start[1], start[0])
    end = list(map(int, c[2].split()))
    enddate = datetime.datetime(end[2], end[1], end[0])
    numyears = (enddate - startdate).days/365
    chambers[name] = Chamber(name, startdate, enddate, numyears)


cabinets_input = [
    # "Balkenende III,07 07 2006,22 02 2007,CDA VVD",
    # "Balkenende IV,22 02 2007,14 10 2010,CDA PvdA ChristenUnie",
    "Balkenende IV,18 09 2008,13 10 2010,20 02 2010,CDA PvdA ChristenUnie",
    "Rutte I,14 10 2010,4 11 2012,23 04 2012,VVD CDA PVV",
    "Rutte II,5 11 2012,25 10 2017,14 03 2017,VVD PvdA",
    "Rutte III,26 10 2017,09 01 2022,15 01 2022,CDA VVD D66 ChristenUnie",
    "Rutte IV,10 01 2022, 01 01 2024,07 07 2023,VVD D66 CDA ChristenUnie"]

cabinets = {}
for c in cabinets_input:
    c = c.split(',')
    name = c[0]
    start = list(map(int, c[1].split()))
    startdate = datetime.datetime(start[2], start[1], start[0])
    end = list(map(int, c[2].split()))
    enddate = datetime.datetime(end[2], end[1], end[0])
    demissionair = list(map(int, c[3].split()))
    demissionair_date = datetime.datetime(demissionair[2], demissionair[1], demissionair[0])
    coalition = tuple(c[4].split())
    cabinets[name] = Cabinet(name, startdate, enddate, demissionair_date, coalition)
cabinets, chambers

({'Balkenende IV': Cabinet(name='Balkenende IV', start=datetime.datetime(2008, 9, 18, 0, 0), end=datetime.datetime(2010, 10, 13, 0, 0), demissionair=datetime.datetime(2010, 2, 20, 0, 0), coalition=('CDA', 'PvdA', 'ChristenUnie')),
  'Rutte I': Cabinet(name='Rutte I', start=datetime.datetime(2010, 10, 14, 0, 0), end=datetime.datetime(2012, 11, 4, 0, 0), demissionair=datetime.datetime(2012, 4, 23, 0, 0), coalition=('VVD', 'CDA', 'PVV')),
  'Rutte II': Cabinet(name='Rutte II', start=datetime.datetime(2012, 11, 5, 0, 0), end=datetime.datetime(2017, 10, 25, 0, 0), demissionair=datetime.datetime(2017, 3, 14, 0, 0), coalition=('VVD', 'PvdA')),
  'Rutte III': Cabinet(name='Rutte III', start=datetime.datetime(2017, 10, 26, 0, 0), end=datetime.datetime(2022, 1, 9, 0, 0), demissionair=datetime.datetime(2022, 1, 15, 0, 0), coalition=('CDA', 'VVD', 'D66', 'ChristenUnie')),
  'Rutte IV': Cabinet(name='Rutte IV', start=datetime.datetime(2022, 1, 10, 0, 0), end=datetime.datetime(2024, 1, 1, 0, 0), dem

In [66]:
import datetime
def get_period(entities, date):
    # date = datetime.datetime.strptime(date_string[:10], '%Y-%m-%d')
    # date = date.replace(tzinfo=None)
    for c in entities.values():
        if ((c.start <= date) or (date < datetime.datetime(2009,1,1))) and c.end >= date:
            return c.name
    else:
        return 'Error'


def get_demissionair(cabinets, date):
    # date = datetime.datetime.strptime(date_string[:10], '%Y-%m-%d')
    c = get_period(cabinets, date)
    # date = date.replace(tzinfo=None)

    return False if date < cabinets[c].demissionair else True
    
df['Kabinet'] = [get_period(cabinets, d) for d in df['Datum']]
df['Demissionair'] = [get_demissionair(cabinets, d) for d in df['Datum']]
df['Kamer'] = [get_period(chambers, d) for d in df['Datum']]
df['Kamer'].value_counts(), df['Kabinet'].value_counts()

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2788\2701059576.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Kabinet'] = [get_period(cabinets, d) for d in df['Datum']]
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2788\2701059576.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Demissionair'] = [get_demissionair(cabinets, d) for d in df['Datum']]
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_2788\2701059576.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `

(Kamer
 Rutte III        12178
 Rutte II         10761
 Rutte IV          9008
 Rutte I           4766
 Balkenende IV     2532
 Name: count, dtype: int64,
 Kabinet
 Rutte III        14362
 Rutte II         11471
 Rutte IV          6018
 Rutte I           4692
 Balkenende IV     2702
 Name: count, dtype: int64)

In [67]:
INCLUDE_PDF = True

# sort columns
column_list = df.columns.values # current columns
column_order = ['Titel','AgendapuntOnderwerp','Onderwerp','Datum','GestartOp','Vergaderjaar','Jaar','Indiener_persoon', 'Indienende_partij','Indienende_persoon_partij','Kamer', 'Kabinet','Demissionair','StemmingsSoort','BesluitSoort','BesluitTekst','doc_Id','Volgnummer', 'Voor', 'Tegen', 'Delta', 'Sum', 'Text'] # desired columns

not_in_column_order = set(column_list) - set(column_order)
 # don't throw away columns
column_order += sorted(not_in_column_order)
df = df[column_order]

In [69]:

cnt = 0
def process_text(text):
    idx = text.find('De Kamer')
    global cnt
    if idx != -1:
        cnt += 1
    return text if idx == -1 else text[idx:]

def remove_kst(text):
    idx = text.find('kst-')
    return text if idx == -1 else text[:idx]

def find_end(text):
    idx = text.find(', en gaat over tot')
    return text if idx == -1 else text[:idx] + '.'

df['Text'] = df['Text'].fillna('')
df['ClippedText'] = df['Text']
# if you remove the begin part you are removing info about the subject and ministery
# df['ClippedText'] = df['Text'].apply(process_text)
# df['ClippedText'].apply(process_text)
df['ClippedText'] = df['ClippedText'].str.replace('\n', ' ')
df['ClippedText'] = df['ClippedText'].str.replace(r'- [^e]', '', regex=True)
df['ClippedText'] = df['ClippedText'].apply(remove_kst)
df['ClippedText'] = df['ClippedText'].apply(find_end)


In [70]:
#df = df[df['Onderwerp'].str.contains('klimaat')]
import pickle
with open('moties_processed_df.pickle', 'wb') as handle:
    pickle.dump(df.copy(), handle, protocol=pickle.HIGHEST_PROTOCOL)

### Making data ready for ChatGPT

In [71]:
import pickle
import pandas as pd
file = open("moties_processed_df.pickle","rb")
df = pickle.load(file)
df = df[df['Kamer']=='Rutte IV']
df['Text'].str.len().sum()


8940052

In [72]:
df.Indienende_partij.value_counts()


Indienende_partij
SP                  927
PVV                 833
D66                 756
VVD                 746
GroenLinks          663
CDA                 638
PvdA                613
PvdD                599
DENK                507
Groep Van Haga      500
ChristenUnie        475
BBB                 427
SGP                 279
FVD                 220
JA21                214
Volt                173
Omtzigt             157
Fractie Den Haan    138
BIJ1                125
Gündogan             14
50PLUS                4
Name: count, dtype: int64

In [73]:
import string
partij2idx = {p: 'PARTIJ_' + string.ascii_uppercase[idx] for idx, p in enumerate(df.Indienende_partij.unique())}
df['Prompt'] = 'De volgende motie is ingediende door ' + df['Indienende_partij'].apply(lambda x: partij2idx[x]) + ' \n' + df['ClippedText']


In [74]:
df.drop(df[df['ClippedText']==""].index, inplace=True)
with open('moties_for_llm.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)